In [4]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

okt = Okt()

# 감정 사전 구성
sentiment_dict = {
    '긍정': ['좋','재미','쾌적','즐겁','즐기','즐거운','즐겨','즐겼','볼거리가 많','볼거리가많','재밌',
           '멋지다','멋진','신난','신나는','풍부','호강','추천','편리','완벽','우수','기대이상','기대 이상'],
    '부정': ['싫','별로','멀다','춥다','비싸','더럽','볼거리가없','부족','혼잡','난해한', '난해했',
           '실망','시끄러', '시끄럽','시끌','비싸','비싼','아쉽','아쉬운','아쉬움','아쉬웠','지루','짜증','기대이하', '기대 이하']
}

def calculate_sentiment_score(tokens):
    sentiment_score = 0
    for token in tokens:
        for sentiment, words in sentiment_dict.items():
            if token in words:
                if sentiment == '긍정':
                    sentiment_score += 1
                elif sentiment == '부정':
                    sentiment_score -= 1
    return sentiment_score

festival_list = [
     '강릉커피축제'
    ,'대구약령시한방문화축제'
    ,'대구치맥페스티벌'
    ,'밀양아리랑축제'
    ,'봉화은어축제'
    ,'산청한방약초축제'
    ,'수원화성문화제'
    ,'순창장류축제'
    ,'시흥갯골축제'
    ,'안성바우덕이축제'
    ,'여주오곡나루축제'
    ,'연천구석기축제'
    ,'울산옹기축제'
    ,'음성품바축제'
    ,'인천펜타포트음악축제'
    ,'임실N치즈축제'
    ,'진안홍삼축제'
    ,'춘천마임축제'
    ,'한산모시축제'
    ,'횡성한우축제'
]

for festival in festival_list:
    column_headers = ['text', 'label']
    file_name = festival + ".txt"

    # Train 데이터 불러오기
    train_data = pd.read_csv(file_name, names=column_headers, delimiter='\t')

    # 한글 및 공백을 제외한 문자 제거
    train_data['text'] = train_data['text'].apply(lambda x: re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', x))

    # 형태소 분석 수행하여 토큰화
    train_data['tokens'] = train_data['text'].apply(lambda x: okt.morphs(x))

    # 학습 데이터와 테스트 데이터로 분할
    text = train_data['text']
    
    # 감성 점수 계산
    train_data['sentiment_score'] = train_data['tokens'].apply(lambda x: calculate_sentiment_score(x))

    # 긍정/부정 클래스의 샘플이 모두 있는지 확인
    if all(train_data['sentiment_score'] > 0) or all(train_data['sentiment_score'] < 0):
        print(f"Skipping festival {festival}: Not enough samples for both positive and negative classes")
        print("----------------------------------------")
        continue

    # 감성 점수에 따라 긍정/부정 레이블 생성
    train_data['label'] = train_data['sentiment_score'].apply(lambda x: '긍정' if x > 0 else '부정')

    label = train_data['label']

    print(train_data)
    if len(train_data) > 0:
        train_x, test_x, train_y, test_y = train_test_split(text, label, test_size=0.2, random_state=0)

        # TF-IDF 벡터화
        vectorizer = TfidfVectorizer()
        train_vectors = vectorizer.fit_transform(train_x)

        # 로지스틱 회귀 모델 학습
        classifier = LogisticRegression()
        classifier.fit(train_vectors, train_y)

        # 테스트 데이터에 대한 예측 수행
        test_vectors = vectorizer.transform(test_x)
        predictions = classifier.predict(test_vectors)

        # 성능 평가
        accuracy = accuracy_score(test_y, predictions)
        precision = precision_score(test_y, predictions, average='weighted')
        recall = recall_score(test_y, predictions, average='weighted')
        f1 = f1_score(test_y, predictions, average='weighted')

        # 결과 출력
        print("Festival:", festival)
        print("total sentiment score:",train_data['sentiment_score'].sum())
        print("Accuracy:", accuracy)
        print("Precision:", precision)
        print("Recall:", recall)
        print("F1 Score:", f1)
        print("----------------------------------------")
    else:
        print("Missing data in the file")
        
    # 각 축제의 감정 점수 계산 및 저장할 딕셔너리 생성
festival_scores = {}

for festival in festival_list:
    column_headers = ['text', 'label']
    file_name = festival + ".txt"

    # Train 데이터 불러오기
    train_data = pd.read_csv(file_name, names=column_headers, delimiter='\t')

    # 한글 및 공백을 제외한 문자 제거
    train_data['text'] = train_data['text'].apply(lambda x: re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', x))

    # 형태소 분석 수행하여 토큰화
    train_data['tokens'] = train_data['text'].apply(lambda x: okt.morphs(x))

    # 감성 점수 계산
    train_data['sentiment_score'] = train_data['tokens'].apply(lambda x: calculate_sentiment_score(x))

    # 각 축제의 감정 점수 총합 계산
    total_sentiment_score = train_data['sentiment_score'].sum()

    # 딕셔너리에 감정 점수 총합 저장
    festival_scores[festival] = total_sentiment_score

# 감정 점수에 따라 축제를 내림차순으로 정렬하여 순위 출력
sorted_scores = sorted(festival_scores.items(), key=lambda x: x[1], reverse=True)
for rank, (festival, score) in enumerate(sorted_scores, start=1):
    print(f"Rank {rank}: Festival {festival}, Total Sentiment Score: {score}")


                                                 text label  \
0                                       강릉커피축제 아레나       부정   
1     안녕하세요  커피축제에 다녀왔는데요 살짝쿵 남겨보려 합니다  평창 휘닉스에서 토...    부정   
2                                       강릉커피축제 당일치기      부정   
3    강릉커피축제 다녀왔어요 당일치기로 후다닥 다녀왔는데 이번 축제는 입장객 사전 등록...    부정   
4                           강릉여행  시나미 명주 나들이 강릉커피축제      부정   
..                                                ...   ...   
76      강릉커피축제   강릉아레나밎 강릉일원  마지막날 여행이라 축제장은 패스   ...    부정   
77                                       안내  강릉커피축제      부정   
78   안녕하세요 하리오 블로그지기 입니다 오랜만에 재미있는 소식을 전하고자 합니다  금...    긍정   
79                    자연에버리다와 함께한 강릉커피축제 친환경 축제로 호평      부정   
80   친환경 테마 개최 성황리 폐막생분해 종이컵 사용 개인텀블러 지참 유도 사용된 컵은...    부정   

                                               tokens  sentiment_score  
0                                   [강릉, 커피, 축제, 아레나]                0  
1   [안녕하세요, 커피, 축제, 에, 다녀왔는데요, 살짝, 쿵, 남겨, 보려, 합니다,...                0  
2                       

d:\mkm\bigdata\training\bigdata_class\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                 text label  \
0                             월 대구 행사  대구약령시 한방문화축제      부정   
1      매년 월이면 약령시가 떠들썩합니다 바로 개장 주년을 맞은 대구약령시의 한방문화...    긍정   
2                           대구 약령시 한방문화축제 무대 공연 사회자      부정   
3         행사 진행 순서  오늘의 진행 순서는 사전 안내 멘트로 관객분들께 오늘의...    부정   
4                            대구약령시 한방문화축제를 즐기러 다녀오다      부정   
..                                                ...   ...   
73   대구약령시 한방문화축제 구경어제 토요일 날씨가 하도 포근해서 덥기까지 한 날 지인...    부정   
74                                       대구약령시 한방문화축제    부정   
75   약령시로  떠나는 건강한 소풍   대구약령시 한방문화축제  화   토 대구약령시 ...    긍정   
76                                    대구약령시한방문화축제대구여행    부정   
77  대구약령시한방문화축제대구여행  지금 대구에서는  대구약령시한방문화축제가 열리고 있다...    부정   

                                               tokens  sentiment_score  
0                  [월, 대구, 행사, 대구, 약령시, 한, 방문, 화, 축제]                0  
1   [매년, 월, 이, 면, 약령시, 가, 떠들썩합니다, 바로, 개장, 주년, 을, 맞...                3  
2                [대구, 약령

d:\mkm\bigdata\training\bigdata_class\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                 text label  \
0                        대구 치맥 페스티벌 라인업 및 팬미팅 당첨 후기      부정   
1    야호  올해 축제 첫 스타트 다만 현장은 아니고 랜선 참석  내 고향 대구는 맛과...    부정   
2                          대구 치맥 페스티벌 후기지도 라인업 주차정보      부정   
3    대구에 치맥 페스티벌을 보러 타지에서 왔답니다 치맥 페스티벌이 있는줄 이번에 처음...    긍정   
4                     대구 치맥 페스티벌 라인업 가수 시간 가격 기본정보       부정   
..                                                ...   ...   
76    안녕하세요 본격적인 여름이 시작되면서 더운 여름을 시원하게 보낼 수 있는 핫한 ...    긍정   
77                              대구치맥페스티벌 자원봉사치맥프렌즈편      부정   
78   치맥프렌즈는 내가 처음으로 합격한 대외활동이었지만 기대에는 못 미친 경험이었다  ...    부정   
79                     대구 치맥 페스티벌 오래도록 기다렸던 축제 라인업       부정   
80    대한민국 치킨산업의 중심 대구  전국 유명 치킨프랜차이즈 브랜드의 시작은 곧 대...    긍정   

                                               tokens  sentiment_score  
0                 [대구, 치맥, 페스티벌, 라인업, 및, 팬미팅, 당첨, 후기]                0  
1   [야호, 올해, 축제, 첫, 스타트, 다만, 현장, 은, 아니고, 랜선, 참석, 내...                0  
2                 [대구, 치

...

                                                 text label  \
0                            축제여행 춘천마임축제 불의도시 도깨비난장      부정   
1    아마 년 전에 마지막으로 가본 춘천마임축제 올해는 아수라장은 가지 못하고 도깨비난...    긍정   
2                          년 춘천마임축제 불의도시 도깨비난장 후기        부정   
3    춘천 가볼만한 곳 년 춘천마임축제  불의도시 도깨비난장 춘천 삼악산 호수 케이블카...    부정   
4                   춘천마임축제 물의도시 아수라장  레고랜드  춘천대교분수쇼      부정   
..                                                ...   ...   
77    거리에서 만나는 춘천마임축제   잔여 프로그램 및 도깨비난장 할인 정보  글사진...    부정   
78                         ㅈㅅㅇ푸로젝트    춘천마임축제 도깨비난장       부정   
79     월 일    년만에 열리는 춘천마임축제 도깨비난장 우선 즐긴뒤에  추가로 쓰는...    부정   
80                    월축제  봄과 물의도시 시민들이 함께하는 춘천마임축제      부정   
81   안녕하세요 여러분   프랑스 미모스 마임축제 영국의 런던 마임축제와 더불어 세계 ...    부정   

                                               tokens  sentiment_score  
0               [축제, 여행, 춘천, 마임, 축제, 불의, 도시, 도깨비, 난장]                0  
1   [아마, 년, 전, 에, 마지막, 으로, 가본, 춘천, 마임, 축제, 올해, 는, ...                4  
2                [년, 춘천,

d:\mkm\bigdata\training\bigdata_class\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                 text label  \
0            한산모시축제 한산모시문화제 기본정보 베틀 체험하러 월 충남 여행 가요      부정   
1    모시할매와 함께하는 월 충남 축제 한산모시축제를 여러분께 소개해 드려요  한산모시...    긍정   
2                     서천 한산모시축제 한산모시문화제 기본정보 가수 주차장      부정   
3      삼베와 함께 여름철 최고의 옷감으로 꼽는 모시와 관련된 월 축제가 있었으니 바...    긍정   
4                         서천축제 한산모시문화제 문화에 축제를 더하다       부정   
..                                                ...   ...   
73      서천군가볼만한곳 서천군관광명소 서천 한산모시축제가 펼쳐지는 한산모시관과 한산...    부정   
74                                        한산 모시축제        부정   
75     코로나때문에 못하던  한산모시축제를 올해는 개최를 했나보다 저번주말에 점심먹고...    부정   
76                                여러분 한산모시축제에 놀러오세요      부정   
77   오랜만에 인사드려 죄송합니다 개인사정으로인해  몇일 더 정신이 없을거같아요 이일만...    부정   

                                               tokens  sentiment_score  
0   [한산, 모시, 축제, 한산, 모시, 문화제, 기본, 정보, 베틀, 체험, 하러, ...                0  
1   [모시, 할, 매, 와, 함께, 하는, 월, 충남, 축제, 한산, 모시, 축제, 를...                1  
2      [서천, 한산, 모시, 축제, 

d:\mkm\bigdata\training\bigdata_class\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Rank 1: Festival 울산옹기축제, Total Sentiment Score: 59
Rank 2: Festival 밀양아리랑축제, Total Sentiment Score: 47
Rank 3: Festival 한산모시축제, Total Sentiment Score: 47
Rank 4: Festival 대구치맥페스티벌, Total Sentiment Score: 46
Rank 5: Festival 춘천마임축제, Total Sentiment Score: 44
Rank 6: Festival 시흥갯골축제, Total Sentiment Score: 41
Rank 7: Festival 횡성한우축제, Total Sentiment Score: 38
Rank 8: Festival 안성바우덕이축제, Total Sentiment Score: 37
Rank 9: Festival 봉화은어축제, Total Sentiment Score: 36
Rank 10: Festival 음성품바축제, Total Sentiment Score: 34
Rank 11: Festival 임실N치즈축제, Total Sentiment Score: 34
Rank 12: Festival 진안홍삼축제, Total Sentiment Score: 33
Rank 13: Festival 순창장류축제, Total Sentiment Score: 32
Rank 14: Festival 산청한방약초축제, Total Sentiment Score: 28
Rank 15: Festival 여주오곡나루축제, Total Sentiment Score: 27
Rank 16: Festival 연천구석기축제, Total Sentiment Score: 26
Rank 17: Festival 수원화성문화제, Total Sentiment Score: 22
Rank 18: Festival 대구약령시한방문화축제, Total Sentiment Score: 20
Rank 19: Festival 강릉커피축제, Total Sentiment Score: 19
Rank